In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.font_manager as fm

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.style.use('seaborn-v0_8-whitegrid')  # 使用seaborn样式

# 创建输出目录
output_dir = 'dengue_model_prep'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 创建中文字体对象
font_path = 'C:/Windows/Fonts/simhei.ttf'
if os.path.exists(font_path):
    chinese_font = fm.FontProperties(fname=font_path)
    print("使用SimHei字体文件路径")
else:
    # 回退到rcParams方法
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    chinese_font = None
    print("使用rcParams设置SimHei字体")

# 读取处理过的数据
print("=== 开始加载数据 ===")
df = pd.read_csv('processed_dengue_data.csv')

# 确保日期列为datetime类型
df['Date'] = pd.to_datetime(df['Date'])

# 按地区和日期排序
df = df.sort_values(['Region', 'Date'])

print(f"原始数据形状: {df.shape}")
print(f"数据时间范围: {df['Date'].min()} 至 {df['Date'].max()}")
print("数据加载完成！\n")

使用SimHei字体文件路径
=== 开始加载数据 ===
原始数据形状: (1020, 8)
数据时间范围: 2016-01-01 00:00:00 至 2020-12-01 00:00:00
数据加载完成！



In [2]:
# 1. 创建滞后特征（前1-3个月的病例数）
print("1. 创建滞后特征")

# 按地区分组，创建滞后特征
for lag in range(1, 4):
    # 创建滞后特征名称
    lag_col = f'Lag{lag}_Cases'
    
    # 按地区分组创建滞后特征
    df[lag_col] = df.groupby('Region')['Dengue_Cases'].shift(lag)
    
    print(f"  创建了滞后特征: {lag_col}")

1. 创建滞后特征
  创建了滞后特征: Lag1_Cases
  创建了滞后特征: Lag2_Cases
  创建了滞后特征: Lag3_Cases


In [9]:
# 2. 添加季节性指标（月份的正弦、余弦变换）
print("\n2. 添加季节性指标")

# 将月份转换为周期性特征
df['Month_Sin'] = np.sin(2 * np.pi * df['MonthNum'] / 12)
df['Month_Cos'] = np.cos(2 * np.pi * df['MonthNum'] / 12)

print("  添加了月份的正弦变换: Month_Sin")
print("  添加了月份的余弦变换: Month_Cos")

# 可视化季节性特征
plt.figure(figsize=(12, 6))

# 绘制月份的正弦和余弦变换
months = range(1, 13)
sin_values = np.sin(2 * np.pi * np.array(months) / 12)
cos_values = np.cos(2 * np.pi * np.array(months) / 12)

plt.plot(months, sin_values, 'b-o', label='Sin(月份)')
plt.plot(months, cos_values, 'r-o', label='Cos(月份)')

# 设置图表属性
if chinese_font is not None:
    plt.title('月份的周期性变换', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('值', fontproperties=chinese_font, fontsize=12)
    plt.legend(prop=chinese_font)
else:
    plt.title('月份的周期性变换', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('值', fontsize=12)
    plt.legend()

plt.xticks(months, ['一月', '二月', '三月', '四月', '五月', '六月', 
                   '七月', '八月', '九月', '十月', '十一月', '十二月'], fontproperties=chinese_font)
plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/seasonal_transformations.png', dpi=300)
print(f"  保存图表：{output_dir}/seasonal_transformations.png")
plt.close()


2. 添加季节性指标
  添加了月份的正弦变换: Month_Sin
  添加了月份的余弦变换: Month_Cos
  保存图表：dengue_model_prep/seasonal_transformations.png


In [4]:
# 3. 计算移动平均和趋势指标
print("\n3. 计算移动平均和趋势指标")

# 计算3个月移动平均
df['MA3_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean())

# 计算6个月移动平均
df['MA6_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=6, min_periods=1).mean())

# 计算12个月移动平均（年度趋势）
df['MA12_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=12, min_periods=1).mean())

# 计算指数加权移动平均（赋予近期数据更高权重）
df['EWMA3_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.ewm(span=3, min_periods=1).mean())

# 计算环比增长率（相对于上个月）
df['MoM_Growth'] = df.groupby('Region')['Dengue_Cases'].pct_change()

# 计算同比增长率（相对于去年同月）
df['YoY_Growth'] = df.groupby(['Region', 'MonthNum'])['Dengue_Cases'].pct_change()

print("  创建了3个月移动平均: MA3_Cases")
print("  创建了6个月移动平均: MA6_Cases")
print("  创建了12个月移动平均: MA12_Cases")
print("  创建了3个月指数加权移动平均: EWMA3_Cases")
print("  创建了环比增长率: MoM_Growth")
print("  创建了同比增长率: YoY_Growth")

# 可视化移动平均
# 选择一个样本地区进行可视化
sample_region = 'Region IV-A'  # 在前面分析中发现的高发地区
sample_data = df[df['Region'] == sample_region].copy()

plt.figure(figsize=(14, 7))

# 绘制原始病例数和不同窗口的移动平均
plt.plot(sample_data['Date'], sample_data['Dengue_Cases'], 'k-', alpha=0.5, label='原始病例数')
plt.plot(sample_data['Date'], sample_data['MA3_Cases'], 'b-', label='3个月移动平均')
plt.plot(sample_data['Date'], sample_data['MA6_Cases'], 'g-', label='6个月移动平均')
plt.plot(sample_data['Date'], sample_data['MA12_Cases'], 'r-', label='12个月移动平均')

# 设置图表属性
if chinese_font is not None:
    plt.title(f'{sample_region} 登革热病例移动平均', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('病例数', fontproperties=chinese_font, fontsize=12)
    plt.legend(prop=chinese_font)
else:
    plt.title(f'{sample_region} 登革热病例移动平均', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('病例数', fontsize=12)
    plt.legend()

plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/moving_averages.png', dpi=300)
print(f"  保存图表：{output_dir}/moving_averages.png")
plt.close()

# 计算并填充缺失值
df = df.fillna(0)  # 填充创建特征后的NaN值


3. 计算移动平均和趋势指标
  创建了3个月移动平均: MA3_Cases
  创建了6个月移动平均: MA6_Cases
  创建了12个月移动平均: MA12_Cases
  创建了3个月指数加权移动平均: EWMA3_Cases
  创建了环比增长率: MoM_Growth
  创建了同比增长率: YoY_Growth
  保存图表：dengue_model_prep/moving_averages.png


In [7]:
# 1. 创建"爆发"标签
print("1. 创建爆发标签")

# 按地区计算病例数的95%分位数
percentile_95 = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.quantile(0.95))

# 创建爆发标签（1表示爆发，0表示正常）
df['Outbreak'] = (df['Dengue_Cases'] > percentile_95).astype(int)

# 创建不同严重程度的爆发标签
percentile_75 = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.quantile(0.75))
percentile_90 = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.quantile(0.90))
percentile_99 = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.quantile(0.99))

# 创建多级爆发标记
df['Outbreak_Level'] = 0  # 默认无爆发
df.loc[df['Dengue_Cases'] > percentile_75, 'Outbreak_Level'] = 1  # 轻度爆发
df.loc[df['Dengue_Cases'] > percentile_90, 'Outbreak_Level'] = 2  # 中度爆发
df.loc[df['Dengue_Cases'] > percentile_95, 'Outbreak_Level'] = 3  # 重度爆发
df.loc[df['Dengue_Cases'] > percentile_99, 'Outbreak_Level'] = 4  # 极端爆发

outbreak_counts = df['Outbreak'].value_counts()
print(f"  正常情况(0): {outbreak_counts.get(0, 0)} 记录")
print(f"  爆发情况(1): {outbreak_counts.get(1, 0)} 记录")

level_counts = df['Outbreak_Level'].value_counts().sort_index()
for level, count in level_counts.items():
    level_name = {0: '无爆发', 1: '轻度爆发', 2: '中度爆发', 3: '重度爆发', 4: '极端爆发'}[level]
    print(f"  {level_name}({level}): {count} 记录")

# 重新获取sample_data，确保包含新创建的列
sample_data = df[df['Region'] == sample_region].copy()

# 可视化爆发标签
plt.figure(figsize=(14, 7))

# 绘制样本地区的病例数和爆发阈值
plt.plot(sample_data['Date'], sample_data['Dengue_Cases'], 'b-', label='病例数')
plt.axhline(y=percentile_95[sample_data.index[0]], color='r', linestyle='--', 
            label=f'爆发阈值(95%分位数)')

# 标记爆发点
outbreak_points = sample_data[sample_data['Outbreak'] == 1]
plt.scatter(outbreak_points['Date'], outbreak_points['Dengue_Cases'], 
            color='red', s=50, label='爆发点')

# 设置图表属性
if chinese_font is not None:
    plt.title(f'{sample_region} 登革热爆发标记', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('病例数', fontproperties=chinese_font, fontsize=12)
    plt.legend(prop=chinese_font)
else:
    plt.title(f'{sample_region} 登革热爆发标记', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('病例数', fontsize=12)
    plt.legend()

plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/outbreak_detection.png', dpi=300)
print(f"  保存图表：{output_dir}/outbreak_detection.png")
plt.close()

1. 创建爆发标签
  正常情况(0): 969 记录
  爆发情况(1): 51 记录
  无爆发(0): 765 记录
  轻度爆发(1): 153 记录
  中度爆发(2): 51 记录
  重度爆发(3): 34 记录
  极端爆发(4): 17 记录
  保存图表：dengue_model_prep/outbreak_detection.png


In [8]:
# 2. 准备模型训练数据
print("\n2. 准备模型训练数据")

# 选择用于模型的特征
feature_cols = [
    # 基本特征
    'Year', 'MonthNum', 
    # 滞后特征
    'Lag1_Cases', 'Lag2_Cases', 'Lag3_Cases',
    # 季节性特征
    'Month_Sin', 'Month_Cos',
    # 趋势特征
    'MA3_Cases', 'MA6_Cases', 'MA12_Cases', 'EWMA3_Cases',
    'MoM_Growth', 'YoY_Growth'
]

# 目标变量
target_col = 'Outbreak'

# 按时间分割数据（前80%作为训练集，后20%作为测试集）
# 确保没有数据泄露，使用严格的时间分割
split_date = df['Date'].min() + pd.DateOffset(months=int(df['Date'].nunique() * 0.8))
print(f"  训练/测试分割日期: {split_date}")

# 创建训练集和测试集
train_df = df[df['Date'] < split_date].copy()
test_df = df[df['Date'] >= split_date].copy()

print(f"  训练集大小: {train_df.shape}")
print(f"  测试集大小: {test_df.shape}")

# 准备X（特征）和y（目标）
X_train = train_df[feature_cols].copy()
y_train = train_df[target_col].copy()

X_test = test_df[feature_cols].copy()
y_test = test_df[target_col].copy()

# 处理特征中的极端值和缺失值
# 将增长率中的无穷大替换为0
for col in ['MoM_Growth', 'YoY_Growth']:
    X_train[col] = X_train[col].replace([np.inf, -np.inf], np.nan)
    X_test[col] = X_test[col].replace([np.inf, -np.inf], np.nan)
    
    # 使用0填充NaN值
    X_train[col] = X_train[col].fillna(0)
    X_test[col] = X_test[col].fillna(0)

# 标准化特征
scaler = StandardScaler()
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# 将处理后的数据保存到CSV文件
train_df.to_csv(f'{output_dir}/train_data.csv', index=False)
test_df.to_csv(f'{output_dir}/test_data.csv', index=False)
print(f"  训练数据已保存至: {output_dir}/train_data.csv")
print(f"  测试数据已保存至: {output_dir}/test_data.csv")

# 特征重要性分析
print("\n3. 设计风险评分系统的特征")

# 计算每个特征与爆发的相关性
correlation = {}
for feature in feature_cols:
    # 计算特征与目标变量的相关性
    corr = df[feature].corr(df[target_col])
    correlation[feature] = abs(corr)  # 取绝对值

# 按相关性排序
sorted_correlation = sorted(correlation.items(), key=lambda x: x[1], reverse=True)
print("  特征与爆发的相关性排名:")
for feature, corr in sorted_correlation:
    print(f"    {feature}: {corr:.4f}")

# 创建一个简单的风险评分系统
print("\n  设计风险评分系统")

# 基于特征重要性权重设计评分系统
risk_weights = {
    'Lag1_Cases': 0.3,         # 上个月的病例数权重最高
    'MA3_Cases': 0.2,          # 最近3个月平均
    'Month_Sin': 0.15,         # 季节性因素
    'Month_Cos': 0.15,         # 季节性因素
    'YoY_Growth': 0.1,         # 同比增长
    'MoM_Growth': 0.1          # 环比增长
}

# 创建评分函数
def calculate_risk_score(row):
    score = 0
    # 正常化处理的滞后病例数
    if row['Lag1_Cases'] > 0:
        score += min(10, row['Lag1_Cases'] / 500 * 10) * risk_weights['Lag1_Cases']
    
    # 3个月移动平均
    if row['MA3_Cases'] > 0:
        score += min(10, row['MA3_Cases'] / 600 * 10) * risk_weights['MA3_Cases']
    
    # 季节性因素（7-10月为高风险季节）
    month_score = 5 + 5 * row['Month_Sin']  # 变换到0-10
    score += month_score * risk_weights['Month_Sin']
    
    month_cos_score = 5 - 5 * row['Month_Cos']  # 变换到0-10
    score += month_cos_score * risk_weights['Month_Cos']
    
    # 同比增长
    if row['YoY_Growth'] > 0:
        yoy_score = min(10, row['YoY_Growth'] * 10)
        score += yoy_score * risk_weights['YoY_Growth']
    
    # 环比增长
    if row['MoM_Growth'] > 0:
        mom_score = min(10, row['MoM_Growth'] * 10)
        score += mom_score * risk_weights['MoM_Growth']
    
    return score

# 应用评分函数
df['Risk_Score'] = df.apply(calculate_risk_score, axis=1)

# 定义风险等级
df['Risk_Level'] = pd.cut(
    df['Risk_Score'],
    bins=[0, 2, 4, 6, 8, 10],
    labels=['极低', '低', '中', '高', '极高']
)

# 分析风险等级与实际爆发的关系
risk_outbreak = pd.crosstab(df['Risk_Level'], df['Outbreak'])
print("  风险等级与实际爆发情况对比:")
print(risk_outbreak)

# 计算每个风险等级的爆发比例
risk_outbreak_pct = risk_outbreak.div(risk_outbreak.sum(axis=1), axis=0) * 100
print("\n  各风险等级的爆发比例(%):")
print(risk_outbreak_pct[1].round(2))

# 保存风险评分结果
df[['Region', 'Year', 'Month', 'Dengue_Cases', 'Risk_Score', 'Risk_Level', 'Outbreak']].to_csv(
    f'{output_dir}/risk_scores.csv', index=False)
print(f"  风险评分结果已保存至: {output_dir}/risk_scores.csv")

# 可视化风险评分
plt.figure(figsize=(12, 6))

# 按风险等级统计爆发比例
risk_levels = risk_outbreak_pct.index
outbreak_pct = risk_outbreak_pct[1].values

# 创建条形图
bars = plt.bar(risk_levels, outbreak_pct, color=['green', 'lightgreen', 'yellow', 'orange', 'red'])

# 在条形上标记百分比
for bar, pct in zip(bars, outbreak_pct):
    plt.text(bar.get_x() + bar.get_width()/2, pct + 1, f'{pct:.1f}%', 
             ha='center', va='bottom')

# 设置图表属性
if chinese_font is not None:
    plt.title('各风险等级的爆发比例', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('风险等级', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('爆发比例(%)', fontproperties=chinese_font, fontsize=12)
    # 明确设置x轴刻度标签的字体
    plt.xticks(range(len(risk_levels)), risk_levels, fontproperties=chinese_font)
else:
    plt.title('各风险等级的爆发比例', fontsize=16)
    plt.xlabel('风险等级', fontsize=12)
    plt.ylabel('爆发比例(%)', fontsize=12)

plt.ylim(0, 100)
plt.grid(axis='y')
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/risk_level_outbreak.png', dpi=300)
print(f"  保存图表：{output_dir}/risk_level_outbreak.png")
plt.close()


2. 准备模型训练数据
  训练/测试分割日期: 2020-01-01 00:00:00
  训练集大小: (816, 23)
  测试集大小: (204, 23)
  训练数据已保存至: dengue_model_prep/train_data.csv
  测试数据已保存至: dengue_model_prep/test_data.csv

3. 设计风险评分系统的特征
  特征与爆发的相关性排名:
    EWMA3_Cases: 0.4665
    Lag1_Cases: 0.4311
    MA3_Cases: 0.4286
    YoY_Growth: 0.2793
    MA6_Cases: 0.2312
    Month_Sin: 0.2311
    Lag2_Cases: 0.1931
    MA12_Cases: 0.1570
    MoM_Growth: 0.1447
    MonthNum: 0.1114
    Year: 0.0827
    Month_Cos: 0.0742
    Lag3_Cases: 0.0119

  设计风险评分系统
  风险等级与实际爆发情况对比:
Outbreak      0   1
Risk_Level         
极低           41   0
低           154   0
中           309   3
高           405  40
极高           60   8

  各风险等级的爆发比例(%):
Risk_Level
极低     0.00
低      0.00
中      0.96
高      8.99
极高    11.76
Name: 1, dtype: float64
  风险评分结果已保存至: dengue_model_prep/risk_scores.csv
  保存图表：dengue_model_prep/risk_level_outbreak.png
